In [1]:
import os
 
from sqlalchemy import Column, DateTime, String, Integer, ForeignKey, func, create_engine, Sequence
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

import pandas as pd
import requests

In [2]:
def get_user(i):
    users_list = []
    
    for j in range(i):
        user_data = requests.get('https://randomuser.me/api/').json()['results'][0]
        
        user = User()
        user.name = user_data['name']['first'] + ' '+ user_data['name']['last']
        user.email = user_data['email']
        user.country = user_data['location']['country']
        user.city = user_data['location']['city']
        
        comp = 'IT' if user.country in ['Netherlands', 'France', 'Germany'] else 'Estimating'
        company = session.query(Department).filter_by(name=comp).first()        
        company.user.append(user)
        
        UNI = 'TU Delft' if user.country in ['Canada', 'Australia', 'Netherlands'] else 'TU Twente'
        UNI = session.query(University).filter_by(name=UNI).first() 
        UNI.user.append(user)       
        
        users_list.append(user)
    return users_list

In [3]:
engine = create_engine('sqlite:///:memory:')

In [4]:
Base = declarative_base()

class University(Base):
    __tablename__ = 'universities'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    email_address = Column(String)
    
    user = relationship("User", back_populates="universities")


class Department(Base):
    __tablename__ = 'departments'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    email_address = Column(String)
    
    user = relationship("User", back_populates="departments")
    
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, Sequence('user_id_seq'), primary_key=True)
    name = Column ('name', String(50), unique = True)
    email = Column ('email', String(50))
    city = Column ('city', String(50))
    country = Column ('country', String(50))
    department_name = Column(String(50), ForeignKey('departments.name'))
    university = Column(String(50), ForeignKey('universities.name'))

    departments = relationship("Department", order_by=Department.id, back_populates="user")
    universities = relationship("University", order_by=University.id, back_populates="user")

    


Session = sessionmaker(bind=engine)
Base.metadata.create_all(bind=engine)

session = Session()
session.commit()
session.close()

In [5]:
session = Session()

IT = Department()
IT.email_address = 'bla@bla.com'
IT.name = 'IT'

Estimating = Department()
Estimating.email_address = 'Estimating@bla.com'
Estimating.name = 'Estimating'

TU = University()
TU.email_address = 'TU_delft@bla.com'
TU.name = 'TU Delft'

TUT = University()
TUT.email_address = 'TU_Twente@bla.com'
TUT.name = 'TU Twente'

session.add_all([IT, Estimating, TU,TUT])
session.commit()

In [6]:
%%time
U = get_user(100)

Wall time: 24.6 s


In [7]:
%%time
session.add_all(U)
session.commit()

Wall time: 3 ms


In [8]:
pd.read_sql_table('users', engine, index_col='id')

,name,email,city,country,department_name,university
id,,,,,,
1,Edward Gomez,edward.gomez@example.com,Busselton,Australia,Estimating,TU Delft
2,Arnd Jauch,arnd.jauch@example.com,Greven,Germany,IT,TU Twente
3,Marcelina Stenersen,marcelina.stenersen@example.com,Svorkmo,Norway,Estimating,TU Twente
4,Frederick Reynolds,frederick.reynolds@example.com,Mildura,Australia,Estimating,TU Delft
5,سپهر کریمی,sphr.khrymy@example.com,آبادان,Iran,Estimating,TU Twente
6,Linn Nedrelid,linn.nedrelid@example.com,Langset,Norway,Estimating,TU Twente
7,Cléa David,clea.david@example.com,Caen,France,IT,TU Twente
8,Murat Dalkıran,murat.dalkiran@example.com,Hakkâri,Turkey,Estimating,TU Twente
9,Milagros Gallego,milagros.gallego@example.com,Las Palmas de Gran Canaria,Spain,Estimating,TU Twente


In [9]:
pd.read_sql_table('universities', engine, index_col='id')

,name,email_address
id,,
1,TU Delft,TU_delft@bla.com
2,TU Twente,TU_Twente@bla.com


In [10]:
pd.read_sql_table('departments', engine, index_col='id')

,name,email_address
id,,
1,IT,bla@bla.com
2,Estimating,Estimating@bla.com
